In [1]:
import os
import pickle
import multiprocessing
import itertools

from subprocess import check_output
from pyfm import pylibfm
from scipy.sparse import csr_matrix
from scipy.sparse import vstack

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.preprocessing import normalize
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA, FastICA
from sklearn.linear_model import SGDRegressor, LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold, cross_val_predict, train_test_split
from sklearn.cluster import KMeans

%matplotlib inline

pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_columns = 50
pd.options.display.max_rows = 25


%load_ext autotime

/home/subaevdi/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
%run -i create_df.py

Добавляем OneHotEncoder для категориальных фич
Добавляем PCA ICA
Добавляем пары
Добавляем Кластарезацию по X0
train.shape= (4209, 379) df shape =  (8418, 2027)
feat_ohe
feat_pca
feat_ica
feat_pairs_in_top
feat_clust
time: 4.82 s


In [ ]:
def xgb_fit(X_train, y_train, X_test, columns):
    num_boost_rounds = 1200
    
    xgb_params = { 
        'eta': 0.005,
        'max_depth': 4,
        'subsample': 0.8,
        'min_child_weight': 25,
        'colsample_bytree': 0.8,
        'objective': 'reg:linear',
        'eval_metric': 'rmse',
        'base_score': np.mean(y_train), # base prediction = mean(target)
        'silent': 1,
        'tree_method':'hist',
        'seed':42,
    #    'gamma': 0.05,
        'alpha':0.3
    }

    dtrain = xgb.DMatrix(X_train[columns], y_train)
    dtest = xgb.DMatrix(X_test[columns])
    
    
    # train model
    model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)
    y_pred = model.predict(dtest)
    return y_pred

# Отобор признаков. Удаление с конца
1. Обучить модель
2. Провести кросс валидацию
3. Удалить из списка худший признак
4. Провести кросс валидацию на новом списке
5. Сравнить результат 2. и 4. если есть улучшение, то убрать признак, иначе оставить.
6. Взять следующий худший признак и перейти к шагу 2.

feat_ohe - список всех признаков  
feat_current - текущий список модели  

feat\_import - списко признаков в порядке возрастания imp   
feat_сhecked - проверенные признаки 

In [48]:
columns = feat_ohe

kf = KFold(n_splits=5, shuffle=True, random_state=42)
X = df[ix_train]

num_boost_rounds = 1500
    
xgb_params = { 
    'eta': 0.01,
    'max_depth': 4,
    'subsample': 0.9,
    'min_child_weight': 20,
    'colsample_bytree': 0.8,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'maximize':False,
    'base_score': np.mean(y_train), # base prediction = mean(target)
    'silent': 1,
    'tree_method':'hist',
    'seed':42,
#    'gamma': 0.05,
    'alpha':0.1
}

feat_checked = list()
dtrain = xgb.DMatrix(X[columns], y)
model = xgb.train(xgb_params, dtrain, num_boost_rounds)
t = xgb.cv(xgb_params, dtrain, num_boost_rounds, folds=kf, early_stopping_rounds=10)
rmse = t.iloc[-1,0]

print('CV ', t.iloc[-1,0])
imp = model.get_fscore()
feat_model = list(pd.DataFrame({'feat':list(imp.keys()),  'imp':list(imp.values())},
             ).sort_values('imp')['feat'].values)
feat_current = list(set(feat_model) - set(feat_checked))


while len(feat_current) > 0:
    # 
    dtrain = xgb.DMatrix(X[feat_checked + feat_current], y)
    model = xgb.train(xgb_params, dtrain, num_boost_rounds)
    t = xgb.cv(xgb_params, dtrain, num_boost_rounds, folds=kf, early_stopping_rounds=10)
    rmse = t.iloc[-1,0]
    
    imp = model.get_fscore()
    feat_model = list(pd.DataFrame({'feat':list(imp.keys()),  'imp':list(imp.values())},
                 ).sort_values('imp')['feat'].values)
    feat_current = list(set(feat_model) - set(feat_checked))
    feat = feat_current[0]
    
    # Проверка без признака
    dtrain = xgb.DMatrix(X[feat_checked + feat_current[1:]], y)
    t = xgb.cv(xgb_params, dtrain, num_boost_rounds, folds=kf, early_stopping_rounds=10)
    rmse_new = t.iloc[-1,0]

    if rmse_new <= rmse:
        print(feat, 'del {:.2f} {:.2f} delta = {:.3f}'.format(rmse, rmse_new, rmse - rmse_new))
        feat_current = feat_current[1:]
    else:
        print(feat, 'add {:.2f} {:.2f} delta = {:.3f}'.format(rmse, rmse_new, rmse - rmse_new))
        feat_checked.append(feat)
        feat_current = feat_current[1:]
print("Done!")

/home/subaevdi/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


CV  8.89796566667
ohe_56 add 8.89 8.91 delta = -0.017
ohe_91 del 8.90 8.89 delta = 0.006
ohe_181 add 8.89 8.89 delta = -0.001
ohe_194 del 8.89 8.89 delta = 0.002
ohe_57 add 8.89 8.89 delta = -0.001
ohe_143 del 8.91 8.89 delta = 0.017
ohe_73 add 8.89 8.89 delta = -0.003
ohe_189 add 8.89 8.90 delta = -0.010
ohe_85 del 8.89 8.89 delta = 0.003
ohe_158 del 8.89 8.88 delta = 0.005
ohe_31 add 8.88 8.89 delta = -0.012
ohe_48 del 8.88 8.88 delta = 0.002
ohe_204 add 8.88 8.89 delta = -0.010
ohe_144 add 8.88 8.88 delta = -0.005
ohe_191 add 8.88 8.88 delta = -0.005
ohe_8 add 8.88 8.89 delta = -0.012
ohe_10 add 8.88 9.07 delta = -0.192
ohe_187 add 8.88 8.88 delta = -0.006
ohe_177 add 8.88 8.89 delta = -0.015
ohe_160 add 8.88 8.89 delta = -0.009
ohe_142 add 8.88 8.89 delta = -0.014
ohe_205 del 8.88 8.87 delta = 0.007
ohe_75 add 8.87 8.87 delta = -0.001
ohe_80 add 8.87 8.87 delta = -0.005
ohe_53 add 8.87 8.87 delta = -0.000
ohe_135 add 8.87 8.87 delta = -0.002
ohe_6 add 8.87 8.90 delta = -0.034
ohe_1

In [54]:
model_file = './../tmp/' + 'feat_checked_ohe.pkl'
if not os.path.isfile(model_file):
    print("Savind ohe list...")
    with open(model_file, 'wb') as f:
        pickle.dump(feat_checked, f)

Savind ohe list...
time: 2.94 ms


In [ ]:
columns = feat_numb

kf = KFold(n_splits=5, shuffle=True, random_state=42)
X = df[ix_train]

num_boost_rounds = 1500
    
xgb_params = { 
    'eta': 0.01,
    'max_depth': 4,
    'subsample': 0.9,
    'min_child_weight': 20,
    'colsample_bytree': 0.8,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'maximize':False,
    'base_score': np.mean(y_train), # base prediction = mean(target)
    'silent': 1,
    'tree_method':'hist',
    'seed':42,
#    'gamma': 0.05,
    'alpha':0.1
}

feat_checked = list()
dtrain = xgb.DMatrix(X[columns], y)
model = xgb.train(xgb_params, dtrain, num_boost_rounds)
t = xgb.cv(xgb_params, dtrain, num_boost_rounds, folds=kf, early_stopping_rounds=10)
rmse = t.iloc[-1,0]

print('CV ', t.iloc[-1,0])
imp = model.get_fscore()
feat_model = list(pd.DataFrame({'feat':list(imp.keys()),  'imp':list(imp.values())},
             ).sort_values('imp')['feat'].values)
feat_current = list(set(feat_model) - set(feat_checked))


while len(feat_current) > 0:
    # 
    dtrain = xgb.DMatrix(X[feat_checked + feat_current], y)
    model = xgb.train(xgb_params, dtrain, num_boost_rounds)
    t = xgb.cv(xgb_params, dtrain, num_boost_rounds, folds=kf, early_stopping_rounds=10)
    rmse = t.iloc[-1,0]
    
    imp = model.get_fscore()
    feat_model = list(pd.DataFrame({'feat':list(imp.keys()),  'imp':list(imp.values())},
                 ).sort_values('imp')['feat'].values)
    feat_current = list(set(feat_model) - set(feat_checked))
    feat = feat_current[0]
    
    # Проверка без признака
    dtrain = xgb.DMatrix(X[feat_checked + feat_current[1:]], y)
    t = xgb.cv(xgb_params, dtrain, num_boost_rounds, folds=kf, early_stopping_rounds=10)
    rmse_new = t.iloc[-1,0]

    if rmse_new <= rmse:
        print(feat, 'del {:.2f} {:.2f} delta = {:.3f}'.format(rmse, rmse_new, rmse - rmse_new))
        feat_current = feat_current[1:]
    else:
        print(feat, 'add {:.2f} {:.2f} delta = {:.3f}'.format(rmse, rmse_new, rmse - rmse_new))
        feat_checked.append(feat)
        feat_current = feat_current[1:]
print("Done!")

/home/subaevdi/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


CV  8.370612
X143 del 8.37 8.37 delta = 0.001
X342 add 8.37 8.37 delta = -0.001
X343 del 8.37 8.36 delta = 0.004
X355 add 8.36 8.37 delta = -0.002
X22 add 8.36 8.37 delta = -0.002
X133 add 8.36 8.37 delta = -0.002
X13 add 8.36 8.37 delta = -0.002
X223 add 8.36 8.37 delta = -0.002
X249 add 8.36 8.37 delta = -0.002
X284 add 8.36 8.37 delta = -0.002
X152 add 8.36 8.37 delta = -0.002
X27 add 8.36 8.37 delta = -0.003
X114 del 8.36 8.36 delta = 0.001
X75 add 8.36 8.37 delta = -0.002
X51 del 8.36 8.36 delta = 0.002
X356 add 8.36 8.36 delta = -0.003
X77 add 8.36 8.36 delta = -0.001
X35 del 8.36 8.36 delta = 0.001
X336 add 8.36 8.37 delta = -0.003
X367 add 8.36 8.36 delta = -0.000
X138 add 8.36 8.36 delta = -0.000
X120 del 8.37 8.36 delta = 0.002
X174 add 8.36 8.36 delta = -0.000
X340 add 8.36 8.37 delta = -0.003
X211 add 8.36 8.36 delta = -0.001
X161 add 8.36 8.37 delta = -0.003
X127 add 8.36 8.37 delta = -0.007
X85 add 8.36 8.37 delta = -0.003
X177 add 8.36 8.36 delta = -0.000
X70 add 8.36 8.

In [ ]:
model_file = './../tmp/' + 'feat_checked_numb.pkl'
if not os.path.isfile(model_file):
    print("Savind ohe list...")
    with open(model_file, 'wb') as f:
        pickle.dump(feat_checked, f)

In [ ]:
with open('./../tmp/' + 'feat_checked_numb.pkl', 'rb') as f:
    feat_checked_numb = pickle.load(f)
    
with open('./../tmp/' + 'feat_checked_ohe.pkl', 'rb') as f:
    feat_checked_ohe = pickle.load(f)

In [ ]:
columns = feat_checked_numb + feat_checked_ohe

kf = KFold(n_splits=5, shuffle=True, random_state=42)
X = df[ix_train]

num_boost_rounds = 1500
    
xgb_params = { 
    'eta': 0.01,
    'max_depth': 4,
    'subsample': 0.9,
    'min_child_weight': 20,
    'colsample_bytree': 0.8,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'maximize':False,
    'base_score': np.mean(y_train), # base prediction = mean(target)
    'silent': 1,
    'tree_method':'hist',
    'seed':42,
#    'gamma': 0.05,
    'alpha':0.1
}

feat_checked = list()
dtrain = xgb.DMatrix(X[columns], y)
model = xgb.train(xgb_params, dtrain, num_boost_rounds)
t = xgb.cv(xgb_params, dtrain, num_boost_rounds, folds=kf, early_stopping_rounds=10)
rmse = t.iloc[-1,0]

print('CV ', t.iloc[-1,0])
imp = model.get_fscore()
feat_model = list(pd.DataFrame({'feat':list(imp.keys()),  'imp':list(imp.values())},
             ).sort_values('imp')['feat'].values)
feat_current = list(set(feat_model) - set(feat_checked))


while len(feat_current) > 0:
    # 
    dtrain = xgb.DMatrix(X[feat_checked + feat_current], y)
    model = xgb.train(xgb_params, dtrain, num_boost_rounds)
    t = xgb.cv(xgb_params, dtrain, num_boost_rounds, folds=kf, early_stopping_rounds=10)
    rmse = t.iloc[-1,0]
    
    imp = model.get_fscore()
    feat_model = list(pd.DataFrame({'feat':list(imp.keys()),  'imp':list(imp.values())},
                 ).sort_values('imp')['feat'].values)
    feat_current = list(set(feat_model) - set(feat_checked))
    feat = feat_current[0]
    
    # Проверка без признака
    dtrain = xgb.DMatrix(X[feat_checked + feat_current[1:]], y)
    t = xgb.cv(xgb_params, dtrain, num_boost_rounds, folds=kf, early_stopping_rounds=10)
    rmse_new = t.iloc[-1,0]

    if rmse_new <= rmse:
        print(feat, 'del {:.2f} {:.2f} delta = {:.3f}'.format(rmse, rmse_new, rmse - rmse_new))
        feat_current = feat_current[1:]
    else:
        print(feat, 'add {:.2f} {:.2f} delta = {:.3f}'.format(rmse, rmse_new, rmse - rmse_new))
        feat_checked.append(feat)
        feat_current = feat_current[1:]
print("Done!")

In [ ]:
model_file = './../tmp/' + 'feat_checked_numb_ohe.pkl'
if not os.path.isfile(model_file):
    print("Savind ohe list...")
    with open(model_file, 'wb') as f:
        pickle.dump(feat_checked, f)

In [60]:
feat_checked

['X342',
 'X355',
 'X22',
 'X223',
 'X284',
 'X152',
 'ohe_181',
 'X27',
 'ohe_57',
 'X75',
 'ohe_73',
 'ohe_189',
 'X356',
 'ohe_144',
 'X77',
 'ohe_187',
 'X336',
 'ohe_177',
 'X138',
 'ohe_142',
 'X174',
 'X340',
 'X211',
 'X161',
 'X127',
 'X85',
 'X177',
 'X70',
 'ohe_80',
 'ohe_53',
 'ohe_135',
 'X316',
 'ohe_79',
 'ohe_200',
 'X171',
 'X201',
 'ohe_46',
 'X251',
 'X313',
 'ohe_110',
 'X196',
 'X286',
 'X219',
 'ohe_52',
 'ohe_89',
 'ohe_54',
 'X98',
 'ohe_50',
 'X231',
 'ohe_60',
 'X327',
 'X373',
 'X238',
 'X315',
 'X18',
 'X186',
 'X376',
 'X175',
 'X136',
 'ohe_184',
 'ohe_134',
 'X363',
 'X12',
 'X203',
 'ohe_56',
 'X155',
 'X150',
 'X197',
 'X331',
 'ohe_204',
 'X206',
 'ohe_191',
 'ohe_8',
 'X323',
 'ohe_10',
 'X255',
 'X272',
 'ohe_160',
 'X14',
 'X47',
 'X291',
 'X82',
 'ohe_75',
 'ohe_163',
 'X84',
 'X185',
 'ohe_176',
 'X350',
 'X359',
 'X292',
 'X377',
 'X374',
 'X180',
 'X301',
 'X215',
 'X126',
 'ohe_197',
 'X375',
 'X61',
 'X276',
 'X109',
 'X157',
 'X137',
 'X256'

time: 5.54 ms


In [61]:
dtrain = xgb.DMatrix(X[feat_checked], y)
t = xgb.cv(xgb_params, dtrain, num_boost_rounds, folds=kf, early_stopping_rounds=10)
    

/home/subaevdi/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


time: 6.48 s


In [62]:
t

,test-rmse-mean,test-rmse-std,train-rmse-mean,train-rmse-std
0,12.615624,0.412270,12.609231,0.212080
1,12.553581,0.417373,12.546404,0.210782
2,12.483966,0.419118,12.476195,0.212398
3,12.415727,0.420801,12.407793,0.213853
4,12.348433,0.423057,12.339736,0.214982
5,12.282219,0.424846,12.273081,0.216503
6,12.216516,0.426483,12.206710,0.218414
7,12.151707,0.428483,12.141372,0.220042
8,12.090716,0.432711,12.079088,0.219444
9,12.028369,0.434944,12.015707,0.221106


time: 19.9 ms
